In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# kaggle : https://www.kaggle.com/c/santander-customer-transaction-prediction

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,Activation
from sklearn.model_selection import StratifiedKFold,cross_val_score
from keras.utils.generic_utils import get_custom_objects

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Checking to see the expected submission format
sample = pd.read_csv('../input/sample_submission.csv')

print(sample.head())

train = pd.read_csv('../input/train.csv')
print ("Train dataset has {} rows(samples) with {} columns(features) each.".format(*train.shape))

#test = pd.read_csv('test.csv', index_col='Id')
test = pd.read_csv('../input/test.csv')

print ("Test dataset has {} rows(samples) with {} columns(features) each.".format(*test.shape))

In [ ]:
import matplotlib.pyplot as plt

def plot_hist(dataframe, plt,include_columns,bins=20):
    display(('--------------------Histograms of each Numerical Feature--------------------'))
   
    for col in include_columns:
        #if col in 
        plt.figure(figsize=(10,5))
        display(col)
        plt.hist(dataframe[col].dropna(),bins=bins)
        #plt.gca().yaxis.set_major_formatter(ticker.IndexFormatter(1))
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.title('Histogram of '+ col,size=16, y=1.05)
        plt.grid(True)
        plt.show()

def get_categorical_variables(df,include_variables,exclude_variables,index_variable,label): 
    if include_variables is None: 
        include_variables=list(df.select_dtypes(include=[np.object,np.bool]).columns)
    if index_variable in include_variables:
          include_variables.remove(index_variable)

    if label is not None:
        if label in include_variables :
            if exclude_variables is None:
                 exclude_variables=label
            else:
                 exclude_variables=exclude_variables.append(label)

    if exclude_variables is None:
        include_variables_filtered=include_variables
    else:
        include_variables_filtered=[elem for elem in include_variables if elem not in exclude_variables] 
    return include_variables_filtered;

def get_numerical_variables(df,include_variables,exclude_variables,index_variable,label):
    if include_variables is None:
        include_variables=list(df.select_dtypes(exclude=[np.object,np.bool,np.datetime64]).columns)
    if index_variable in include_variables:
        include_variables.remove(index_variable)
    if label is not None:
        if label in include_variables :
            if exclude_variables is None:
                exclude_variables=label
            else:
                exclude_variables=exclude_variables.append(label)

    if exclude_variables is None:
          include_variables_filtered=include_variables
    else:
          include_variables_filtered=[elem for elem in include_variables if elem not in exclude_variables] 
    return include_variables_filtered;


In [ ]:
index_variable='ID_code'
label='target'
include_numerical_variables=None # if None then we select all except the  exclude_numerical_variables
include_categorical_variables=None # if None then we select all except the  exclude_categorical_variables
exclude_numerical_variables=None
exclude_categorical_variables=None
categorical_variables =get_categorical_variables(train,include_categorical_variables,exclude_categorical_variables,index_variable,label)
numerical_variables=get_numerical_variables(df=train,include_variables=None,exclude_variables=None,index_variable=index_variable,label=label)

display(('=====================Categorical Variables==================='))
display(categorical_variables)
  
display(('=====================Numerical Variables====================='))
display(numerical_variables)
display(('=====================Dependent Variable====================='))
display(label)

In [ ]:
from sklearn import preprocessing

def normalize_data(data):
    #normalize all the columns(features) so that all the values in the column lie between 0 and 1
    #this way each features will get equal preference regardless of their actual range
    n_data=pd.DataFrame()
    n_data = pd.DataFrame(preprocessing.normalize(data),columns=data.columns)
    return n_data


In [ ]:
from sklearn.model_selection import train_test_split
id_code=train[index_variable]
y=train[label]
X=normalized_train=normalize_data(train[numerical_variables])
#X_trast=normalized_train=normalize_data(test[numerical_variables])
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.3)

In [ ]:
X_y_train = pd.concat([X_train,y_train],axis=1)
X_y_train_1s = X_y_train[X_y_train['target'] == 1]

X_y_train_0s = X_y_train[X_y_train['target'] == 0]
#keep_0s = X_y_train_0s.sample(frac=X_y_train_1s.shape[0]/X_y_train_0s.shape[0]) for downsampling
rep_1 =[X_y_train_1s for x in range(X_y_train_0s.shape[0]//X_y_train_1s.shape[0] )]
keep_1s = pd.concat(rep_1, axis=0)
X_y_train = pd.concat([X_y_train_0s,keep_1s],axis=0)
#X_y_train = pd.concat([keep_0s,X_y_train_1s],axis=0)
y_train=X_y_train[label]
X_train=X_y_train[numerical_variables]
X_y_train.groupby('target').size()/train.shape[0]

In [ ]:
#HYPEROPT FOR LGBM
from hyperopt import STATUS_OK
from timeit import default_timer as timer
import lightgbm as lgb

MAX_EVALS = 50
N_FOLDS = 10
ITERATION = 0
# Define the search space
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                 {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}


def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    print ('Params testing: ', params)
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_folds cross validation
    train_set = lgb.Dataset(X_train, y_train)
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 50)
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    pred_auc =model.predict_proba(X_validate, batch_size = 128, verbose = 0)
    acc = roc_auc_score(y_validate, pred_auc)
    print('AUC:', acc)
    sys.stdout.flush() 
   
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

bayes_trials = Trials()
best = fmin(fn=objective,  space=space,  algo=tpe.suggest, max_evals=MAX_EVALS,trials = bayes_trials)

print("Hyperopt estimated optimum {}".format(best))

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import GradientBoostingClassifier

space = {
    'max_depth': hp.uniform('max_depth', 2, 4),
    'n_estimators': hp.uniform('n_estimators', 20, 50),
    'learning_rate': hp.uniform('learning_rate', 0.05, 1.0),
    'max_features': hp.uniform('max_features', 2, 4),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5, 5),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0, 10),
}

def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'n_estimators': '{:.3f}'.format(params['n_estimators']),
        'learning_rate': int(params['learning_rate']),
        'max_features': '{:.3f}'.format(params['max_features']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'min_samples_split': '{:.3f}'.format(params['min_samples_split']),
    }
    
    clf = GradientBoostingClassifier(**params, random_state = 0)
     
    
    score = cross_val_score(clf, X, Y, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score



best = fmin(fn=objective,  space=space, algo=tpe.suggest, max_evals=MAX_EVALS)

print("Hyperopt estimated optimum {}".format(best))

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validate, y_validate)))
    y_pred = gb.predict(X_validate)

    print("Confusion Matrix:")
    print(confusion_matrix(y_validate, y_pred))
    print("Classification Report")
    print(classification_report(y_validate, y_pred))
    
    y_scores_gb = gb.decision_function(X_validate)
    fpr_gb, tpr_gb, _ = roc_curve(y_validate, y_scores_gb)
    roc_auc_gb = auc(fpr_gb, tpr_gb)

    print("Area under ROC curve = {:0.2f}".format(roc_auc_gb))
    print()

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score,accuracy_score,f1_score

y_pred=model.predict_classes(X_validate, batch_size=32, verbose=1)
labels = [0,1]
cm = confusion_matrix(y_validate, y_pred, labels)
display(cm)
class_error_fp=cm[0][1]/(cm[0][0]+cm[0][1])
class_error_fn=cm[1][0]/(cm[1][0]+cm[1][1])

Test_set_accuracy_rate=accuracy_score(y_validate,y_pred)

print ("Test set accuracy:{} Class_error_fp: {} Class_error_fn: {}".format(Test_set_accuracy_rate,class_error_fp,class_error_fn))


In [ ]:
def objective(params):
    params = {'n_estimators': int(params['n_estimators']), 'max_depth': int(params['max_depth'])}
    clf = RandomForestClassifier(n_jobs=4, class_weight='balanced', **params)
    score = cross_val_score(clf, X, Y, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score

space = {
    'n_estimators': hp.quniform('n_estimators', 25, 500, 25),
    'max_depth': hp.quniform('max_depth', 1, 10, 1)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
Gini -0.218 params {'n_estimators': 50, 'max_depth': 1}
Gini -0.243 params {'n_estimators': 175, 'max_depth': 4}
Gini -0.237 params {'n_estimat

In [ ]:
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        learning_rate=0.05,
        n_jobs=4,
        **params
    )
    
    score = cross_val_score(clf, X, Y, scoring=gini_scorer, cv=StratifiedKFold()).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score

space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)